In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

In [3]:
data = yf.download(['EURMXN=X','MXN=X'], start='2020-01-01', end='2025-01-24')['Adj Close']
data.tail()

[*********************100%***********************]  2 of 2 completed


Ticker,EURMXN=X,MXN=X
Date,,
2025-01-17 00:00:00+00:00,21.460199,20.832729
2025-01-20 00:00:00+00:00,21.363501,20.791290
2025-01-21 00:00:00+00:00,21.358101,20.497240
2025-01-22 00:00:00+00:00,21.498199,20.661200
2025-01-23 00:00:00+00:00,21.332600,20.487101


In [6]:
positions = [2400000, 1500000]
conf = 0.95

In [14]:
port = data * positions
port['total'] = port.sum(axis=1)
port['rendimiento'] = port['total'].pct_change().dropna()
port

Ticker,EURMXN=X,MXN=X,total,rendimiento
Date,,,,
2020-01-01 00:00:00+00:00,5.095392e+07,2.837610e+07,7.933002e+07,NaN
2020-01-02 00:00:00+00:00,5.091096e+07,2.834820e+07,7.925916e+07,-0.000893
2020-01-03 00:00:00+00:00,5.051088e+07,2.824545e+07,7.875633e+07,-0.006344
2020-01-06 00:00:00+00:00,5.074656e+07,2.841714e+07,7.916370e+07,0.005173
2020-01-07 00:00:00+00:00,5.063280e+07,2.826217e+07,7.889497e+07,-0.003395
...,...,...,...,...
2025-01-17 00:00:00+00:00,5.150448e+07,3.124909e+07,8.275357e+07,0.018864
2025-01-20 00:00:00+00:00,5.127240e+07,3.118694e+07,8.245934e+07,-0.003556
2025-01-21 00:00:00+00:00,5.125944e+07,3.074586e+07,8.200530e+07,-0.005506
